# Meridonal Overturning circulation
- calculations may be improved by taking partial bottom cells
- possibly SSH adds accuracy, but it is centered on the T-grid

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from MOC import calculate_AMOC, AMOC_max, approx_lats
from grid import create_tdepth
from paths import file_ex_ocn_ctrl, path_results
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_DZ, xr_DXU

In [ ]:
tdepth = create_tdepth('ocn')

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
%%time
AMOC = calculate_AMOC(ds)

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
lats = approx_lats('ocn')
X, Y = np.meshgrid(lats, -tdepth/1e3)
plt.pcolormesh(X, Y, AMOC/1e4, vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()
plt.tight_layout()

In [ ]:
AMOC_max(AMOC)

In [ ]:
%%time
for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run='ctrl', tavg='yrly', name='VVEL')):
    # ca. 20 per year
    print(i, y, s)
    ds = xr.open_dataset(s, decode_times=False)
    AMOC = calculate_AMOC(ds)
    if i==0:
        AMOC_out = AMOC.copy()
    else:
        AMOC_out = xr.concat([AMOC_out, AMOC], dim='time')
#     if y==202: break
AMOC_out.to_netcdf(f'{path_results}/MOC/AMOC_ctrl.nc')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(AMOC_max(AMOC_out).values/1e6, lw=2, label='CTRL')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('AMOC (26$^\circ$N, 1000 m) [Sv]', fontsize=16)
plt.legend(fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_timeseries')